<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em>

# Feature Matching (Merkmalabgleich)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def display(img,cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')

In [ ]:
reeses = cv2.imread('../DATA/reeses_puffs.png',0)     

In [ ]:
display(reeses)

In [ ]:
cereals = cv2.imread('../DATA/many_cereals.jpg',0) 

In [ ]:
display(cereals)

--------
# Brute Force-Erkennung mit ORB Deskriptoren (Descriptors)

In [ ]:
# Initiiere ORB-Detektor
orb = cv2.ORB_create()

# Finde die Schlüsselpunkte (keypoints) und Deskriptoren mit ORB
kp1, des1 = orb.detectAndCompute(reeses,None)
kp2, des2 = orb.detectAndCompute(cereals,None)

# Erzeuge BFMatcher-Objekt
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Vergleiche die Deskriptoren.
matches = bf.match(des1,des2)

# Sortiere sie in der Reihenfolge ihrer Distanz
matches = sorted(matches, key = lambda x:x.distance)

# Zeichne die ersten 25 Treffer.
reeses_matches = cv2.drawMatches(reeses,kp1,cereals,kp2,matches[:25],None,flags=2)

In [ ]:
display(reeses_matches)

# Brute-Force-Erkennung mit SIFT-Deskriptoren und Ratio-Test

In [ ]:
# Erzeuge ein SIFT-Object
sift = cv2.xfeatures2d.SIFT_create()

# Finde die Schlüsselpunkte und Deskriptoren mit SIFT
kp1, des1 = sift.detectAndCompute(reeses,None)
kp2, des2 = sift.detectAndCompute(cereals,None)

# BFMatcher mit Standardparameters
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Wende das Quotientenkriterium (Ratio-Test) an
good = []
for match1,match2 in matches:
    if match1.distance < 0.75*match2.distance:
        good.append([match1])

# cv2.drawMatchesKnn erwartet eine Liste aus Listen als Treffer.
sift_matches = cv2.drawMatchesKnn(reeses,kp1,cereals,kp2,good,None,flags=2)

In [ ]:
display(sift_matches)

# FLANN-basierter Vergleicher (Matcher)

In [ ]:
# Initiiere SIFT-Detektor
sift = cv2.xfeatures2d.SIFT_create()

# Finde die Schlüsselpunkte und Deskriptoren mit SIFT
kp1, des1 = sift.detectAndCompute(reeses,None)
kp2, des2 = sift.detectAndCompute(cereals,None)

# FLANN Parameter
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)  

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

good = []

# Quotientenkriterium (Ratio-Test)
for i,(match1,match2) in enumerate(matches):
    if match1.distance < 0.7*match2.distance:
        
        good.append([match1])


flann_matches = cv2.drawMatchesKnn(reeses,kp1,cereals,kp2,good,None,flags=0)

display(flann_matches)

In [ ]:
# Initiiere SIFT-Detektor
sift = cv2.xfeatures2d.SIFT_create()

# Finde die Schlüsselpunkte und Deskriptoren mit SIFT
kp1, des1 = sift.detectAndCompute(reeses,None)
kp2, des2 = sift.detectAndCompute(cereals,None)

# FLANN Parameter
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)  

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

# Es sollen nur gute Treffer gezeichnet werden, daher wird eine Maske kreiert
matchesMask = [[0,0] for i in range(len(matches))]

# Quotientenkriterium (Ratio-Test)
for i,(match1,match2) in enumerate(matches):
    if match1.distance < 0.7*match2.distance:
        matchesMask[i]=[1,0]

draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = 0)

flann_matches = cv2.drawMatchesKnn(reeses,kp1,cereals,kp2,matches,None,**draw_params)

In [ ]:
display(flann_matches)